In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
raw_dataset = pd.read_csv('../data/preset/minibatch.csv')

raw_dataset

,movie_id,user_id,rate,time,content
0,26825482,莫挨脑子,1.0,2022-03-25 20:24:25,看到影片开头 华谊兄弟几个字的时候 我就心里有数了……果然…..
1,26825482,壹安²,4.0,2022-03-26 12:35:51,一定要看IMAX一定要看IMAX一定要看IMAX！画面和特效远超预期他M的，最后视觉大高潮我...
2,26825482,ZeonGin Sou,1.0,2022-03-26 23:43:12,大概是主创团队致力于讲一个自嗨至极的阴谋论，导致全片的特效竟然没有服务于剧情和刺激，这对于一...
3,26825482,夏小時,2.0,2022-02-03 17:43:26,儘管降低了心理預期，但是還是沒料到這災難片能拍成史詩級災難。能感覺導演應當有致敬駭客帝國的意...
4,26825482,人民南路壹号,2.0,2022-02-13 12:22:24,加上最后一段的灾难特效算它勉强及格吧，在导演本人的作品里应当排在倒数的，很多逻辑漏洞和一言难...
5,26825482,joe_white,4.0,2022-03-25 16:51:01,这才是在电影院的电影，无视定律太过于牛逼减一颗😂
6,26825482,宇星天河,2.0,2022-03-25 12:14:43,【4.0=】建议内宣赶紧收编艾默里奇，筹拍德特里克堡大电影，分形粒子特效用来做疫苗内的纳米机...
7,26825482,玩儿少,3.0,2022-03-25 21:58:42,罗兰的正常水平发挥吧，拍了那么多年灾难片了，题材在改变，但是套路却基本一致，这部倒算是大咖云...
8,26825482,胖球,4.0,2022-03-29 23:14:15,这不还行吗？剧情不算特别无脑，场面确实非常惊艳，就足够了～
9,26825482,亵渎电影,1.0,2022-03-30 17:04:17,编剧写这种东西，本意是在写一个粗制滥造的网大吧，不仅时时刻刻在侮辱观众的智商，还侮辱了外星智...


In [3]:
dataset = raw_dataset.dropna(axis=0).reset_index()

dataset

,index,movie_id,user_id,rate,time,content
0,0,26825482,莫挨脑子,1.0,2022-03-25 20:24:25,看到影片开头 华谊兄弟几个字的时候 我就心里有数了……果然…..
1,1,26825482,壹安²,4.0,2022-03-26 12:35:51,一定要看IMAX一定要看IMAX一定要看IMAX！画面和特效远超预期他M的，最后视觉大高潮我...
2,2,26825482,ZeonGin Sou,1.0,2022-03-26 23:43:12,大概是主创团队致力于讲一个自嗨至极的阴谋论，导致全片的特效竟然没有服务于剧情和刺激，这对于一...
3,3,26825482,夏小時,2.0,2022-02-03 17:43:26,儘管降低了心理預期，但是還是沒料到這災難片能拍成史詩級災難。能感覺導演應當有致敬駭客帝國的意...
4,4,26825482,人民南路壹号,2.0,2022-02-13 12:22:24,加上最后一段的灾难特效算它勉强及格吧，在导演本人的作品里应当排在倒数的，很多逻辑漏洞和一言难...
5,5,26825482,joe_white,4.0,2022-03-25 16:51:01,这才是在电影院的电影，无视定律太过于牛逼减一颗😂
6,6,26825482,宇星天河,2.0,2022-03-25 12:14:43,【4.0=】建议内宣赶紧收编艾默里奇，筹拍德特里克堡大电影，分形粒子特效用来做疫苗内的纳米机...
7,7,26825482,玩儿少,3.0,2022-03-25 21:58:42,罗兰的正常水平发挥吧，拍了那么多年灾难片了，题材在改变，但是套路却基本一致，这部倒算是大咖云...
8,8,26825482,胖球,4.0,2022-03-29 23:14:15,这不还行吗？剧情不算特别无脑，场面确实非常惊艳，就足够了～
9,9,26825482,亵渎电影,1.0,2022-03-30 17:04:17,编剧写这种东西，本意是在写一个粗制滥造的网大吧，不仅时时刻刻在侮辱观众的智商，还侮辱了外星智...


In [4]:
val_size = 0.25

idx = list(range(len(dataset)))

random.shuffle(idx)

dataset_train_idx, dataset_val_idx = idx[int(val_size * len(idx)):], idx[:int(val_size * len(idx))]

dataset_train, dataset_val = dataset.iloc[dataset_train_idx], dataset.iloc[dataset_val_idx]

dataset_train

,index,movie_id,user_id,rate,time,content
14,14,26825482,安之炼金师,2.0,2022-02-12 14:46:11,文戏烂到爆
16,16,26825482,老罗Lazzaro,1.0,2022-04-01 01:07:37,无趣灾难，扯出天际，超越科幻范畴的奇幻剧本，空壳式角色设定充满着各种临时起意和罔顾逻辑。
11,11,26825482,火龙果不耐受者,1.0,2022-03-26 17:37:02,WTF！什么鬼！烂的一塌糊涂！
4,4,26825482,人民南路壹号,2.0,2022-02-13 12:22:24,加上最后一段的灾难特效算它勉强及格吧，在导演本人的作品里应当排在倒数的，很多逻辑漏洞和一言难...
17,17,26825482,✔,1.0,2022-03-30 22:50:10,不意外，过气灾难片路数。大概是全人类得新冠之后大脑缩小了一圈。✘/0
12,12,26825482,Keith Lee,3.0,2022-03-25 21:17:34,时间过得真快，不敢相信看罗兰艾默里奇上一部《Midway》已经是两年前的事情了。这次罗兰艾默...
10,10,26825482,JoeyLu陆柏宇,1.0,2022-03-27 17:02:23,难看的一批
13,13,26825482,Henrique Asano,3.0,2022-03-25 22:06:30,脑洞大开，硬塞中国元素是很过分了，当然还是适合银幕看看的无脑爽片
3,3,26825482,夏小時,2.0,2022-02-03 17:43:26,儘管降低了心理預期，但是還是沒料到這災難片能拍成史詩級災難。能感覺導演應當有致敬駭客帝國的意...
1,1,26825482,壹安²,4.0,2022-03-26 12:35:51,一定要看IMAX一定要看IMAX一定要看IMAX！画面和特效远超预期他M的，最后视觉大高潮我...
